# Libraries

In [1]:
# Libraries
#import pokebase as pb
import requests
import json
import pandas as pd
import numpy as np

# Collect the Data

In [2]:
query = requests.get('https://pokeapi.co/api/v2/move?limit=1000&offset=0')
query_data = json.loads(query.text)
print(query_data.keys())

dict_keys(['count', 'next', 'previous', 'results'])


In [3]:
print(len(query_data["results"]))

844


In [4]:
move_data = query_data["results"]

# Data Exploration

In [5]:
test = move_data[10]
# print(test.keys())
print(test["name"])

move_query = requests.get(test["url"])
move_query_data = json.loads(move_query.text)
print(move_query_data.keys())

vice-grip
dict_keys(['accuracy', 'contest_combos', 'contest_effect', 'contest_type', 'damage_class', 'effect_chance', 'effect_changes', 'effect_entries', 'flavor_text_entries', 'generation', 'id', 'learned_by_pokemon', 'machines', 'meta', 'name', 'names', 'past_values', 'power', 'pp', 'priority', 'stat_changes', 'super_contest_effect', 'target', 'type'])


In [6]:
print(move_query_data["meta"])

{'ailment': {'name': 'none', 'url': 'https://pokeapi.co/api/v2/move-ailment/0/'}, 'ailment_chance': 0, 'category': {'name': 'damage', 'url': 'https://pokeapi.co/api/v2/move-category/0/'}, 'crit_rate': 0, 'drain': 0, 'flinch_chance': 0, 'healing': 0, 'max_hits': None, 'max_turns': None, 'min_hits': None, 'min_turns': None, 'stat_chance': 0}


In [7]:
move_query_data["effect_changes"]

[]

In [8]:
move_query_data["generation"]

{'name': 'generation-i', 'url': 'https://pokeapi.co/api/v2/generation/1/'}

# Data Cleaning
**NOTES**
- need to heck the `past_values` key to makes ure that the data we are getting for those things is up to date with the infomration we want
- need to substitue `effect_chance` info `effect_entries`

In [9]:
# handy variables
colums_needed = ["name", "accuracy", "class", "game_desc", "effect_chance", "generation", "machines", "crit_rate", "type", "power", "pp", "priority"]
# gens = ['generation-i', 'generation-ii', 'generation-iii', 'generation-iv', 'generation-v']
gens = {'generation-i':1, 'generation-ii':2, 'generation-iii':3, 'generation-iv':4, 'generation-v':5}
latest_game = "back-2-white-2"
second_latest_game = "black-white"

In [10]:
print(len(move_data))

844


In [11]:
list(gens.keys())

['generation-i',
 'generation-ii',
 'generation-iii',
 'generation-iv',
 'generation-v']

In [12]:
# make the dataframe
moves_df = pd.DataFrame(columns=colums_needed)

for move in move_data:
    # pull the page data that we need
    move_query = requests.get(move["url"])
    move_query_data = json.loads(move_query.text)
    # print(move_query_data["generation"])
    if move_query_data["generation"]["name"] in list(gens.keys()): # only get the abilities from the first 5 generation
        # print("got here")
        # print("here")
        temp = [""] * len(colums_needed)

        temp[0] = move_query_data["name"] # get the move's name

        temp[1] = move_query_data["accuracy"] # get the accuracy of the move
        
        temp[2] = move_query_data["damage_class"]["name"] # get the damgage type of the class
        
        effect_data = move_query_data["effect_entries"] # get the in game description of the move
        for text in effect_data:
            if text["language"]["name"] == "en": # get the english descriptions from black and white
                desc = text["short_effect"]
                desc = desc.replace('$effect_chance', str(move_query_data["effect_chance"]))
                temp[3] = desc
                
                
                desc2 = text["effect"].replace('$effect_chance', str(move_query_data["effect_chance"]))
                # print(desc2)
                temp[4] = desc2
                break
                
        # get the generation the move is from
        temp[5] = gens[move_query_data["generation"]["name"]]
        # print(temp[0], "=", temp[5])
                    
        moves_df.loc[len(moves_df.index)] = temp

#         if "iv" in gen:
#             temp[1] = 4
#         elif "v" in gen:
#             temp[1] = 5
#         elif "iii" in gen:
#             temp[1] = 3
#         elif "ii" in gen:
#             temp[1] = 2
#         elif "i" in gen:
#             temp[1] = 1
#         else: # raise an error if we get a value that is not one of these generations
#             print(gen)
#             raise ValueError("We got an invalid generation!")

#         effect_data = move_query_data["effect_entries"] # get the effect description
#         for text in effect_data:
#             if text["language"]["name"] == "en": # get the english descriptions
#                     temp[2] = text["effect"]
#                     break

        

        # print(temp)

        # moves_df.loc[len(moves_df.index)] = temp                

In [13]:
moves_df

,name,accuracy,class,game_desc,effect_chance,generation,machines,crit_rate,type,power,pp,priority
0,pound,100,physical,Inflicts regular damage with no additional eff...,Inflicts regular damage.,1,,,,,,
1,karate-chop,100,physical,Has an increased chance for a critical hit.,Inflicts regular damage. User's critical hit ...,1,,,,,,
2,double-slap,85,physical,Hits 2-5 times in one turn.,Inflicts regular damage. Hits 2–5 times in on...,1,,,,,,
3,comet-punch,85,physical,Hits 2-5 times in one turn.,Inflicts regular damage. Hits 2–5 times in on...,1,,,,,,
4,mega-punch,85,physical,Inflicts regular damage with no additional eff...,Inflicts regular damage.,1,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
572,shadow-hold,None,status,Prevents the target from leaving battle.,The target cannot switch out normally. Ignore...,3,,,,,,
573,shadow-mist,100,status,Lowers the target's evasion by two stages.,Lowers the target's evasion by two stages.,3,,,,,,
574,shadow-panic,90,status,Confuses the target.,Confuses the target.,3,,,,,,
575,shadow-shed,None,status,"Removes Light Screen, Reflect, and Safeguard.","Removes the effects of light screen, reflect, ...",3,,,,,,


In [14]:
moves_df.loc[1, "game_desc"]

'Has an increased chance for a critical hit.'

In [15]:
moves_df.loc[1, "effect_chance"]

"Inflicts regular damage.  User's critical hit rate is one level higher when using this move."

In [18]:
move_names = moves_df[["name"]].copy()
move_names

,name
0,pound
1,karate-chop
2,double-slap
3,comet-punch
4,mega-punch
...,...
572,shadow-hold
573,shadow-mist
574,shadow-panic
575,shadow-shed


In [19]:
move_names.to_csv("move_names.csv")